# Manipulating inputs & output(操作输入输出)
RunnableParallel 可以用于操作一个 Runnable 的输出，使其与序列中下一个 Runnable 的输入格式匹配。

这里预期 prompt 的输入应该是一个带有“context”和“question”键的映射。用户输入只是问题。因此，我们需要使用我们的检索器获取上下文，并将用户输入传递到“question”键下。

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores.faiss import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough,RunnableParallel

# 创建向量库
vectorstore = FAISS.from_texts(
  ["小王昨天去了北京，后天回上海","小李昨天去了上海，后天回北京"],
  embedding=OpenAIEmbeddings(

  )
)
retriever=vectorstore.as_retriever()

# 提示词模板String
template="""只回答关于Context里内容的问题。如果在Context内容未找到答案将如实告诉用户：抱歉我还未学习该知识，请谅解。:
Context: {context}

Question: {question}
"""

# 创建提示词模板
prompt = ChatPromptTemplate.from_template(template)

# 创建AI模型
model = ChatOpenAI(

)

retriever_chain = (
  {"context":retriever,"question":RunnablePassthrough()}
  | prompt
  | model
  | StrOutputParser()
)

retriever_chain.invoke("小王昨天去哪了")


'小王昨天去了北京。'

请注意，当使用RunnableParallel与另一个Runnable组合时，我们甚至不需要将字典包装在RunnableParallel类中 - 类型转换已经为我们处理了。在链的上下文中，这两者是等价的：

In [4]:
{"context": retriever, "question": RunnablePassthrough()}
RunnableParallel({"context": retriever, "question": RunnablePassthrough()}) | prompt | model | StrOutputParser()
RunnableParallel(context=retriever, question=RunnablePassthrough())

{
  context: VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001D2B4F80C40>),
  question: RunnablePassthrough()
}

### 并行化操作(Parallelize steps)
RunnableParallel（又名RunnableMap）使得在并行执行多个Runnables，并将这些Runnables的输出作为一个映射返回变得容易。

In [5]:
where_chain = ChatPromptTemplate.from_template("告诉我 {topic} 在哪里") | model 
play_chain = ChatPromptTemplate.from_template("告诉我 {topic} 有什么好玩的") | model  

map_chain = RunnableParallel(where=where_chain, play=play_chain)

map_chain.invoke({"topic": "北京"})


{'where': AIMessage(content='北京是中国的首都，位于中国北部，地处华北平原的西北部。'),
 'play': AIMessage(content='北京有许多著名的景点和地标，例如故宫、天安门广场、颐和园、长城等。此外，你还可以到798艺术区欣赏当代艺术作品，游览南锣鼓巷的胡同文化，品尝北京烤鸭和老北京火锅，体验京剧表演等。北京还有许多博物馆、公园和历史文化街区等值得一游。')}